In [ ]:
#ID3 - https://pypi.org/project/chefboost/0.0.1/
#https://towardsdatascience.com/chefboost-an-alternative-python-library-for-tree-based-models-f46af028a348/
%pip install tqdm==4.30.0
%pip install chefboost

from chefboost import Chefboost as chef
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier



import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tqdm>=4.64.1, but you have tqdm 4.30.0 which is incompatible.
prophet 1.1.6 requires tqdm>=4.36.1, but you have tqdm 4.30.0 which is incompatible.
huggingface-hub 0.30.1 requires tqdm>=4.42.1, but you have tqdm 4.30.0 which is incompatible.
spacy 3.8.5 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.30.0 which is incompatible.


In [17]:
path = "/content/Congressional Voting Records/house-votes-84.data"
column_names = ['ClassName', 'HandicappedInfants', 'WaterProjectCostSharing', 'AdoptionOfTheBudgetResolution', 'PhysicianFeeFreeze', 'ElSalvadorAid', 'ReligiousGroupsInSchools', 'AntiSatelliteTestBan', 'AidToNicaraguanContracts', 'MxMissile', 'Immigration', 'SynfuelsCorporationCutback', 'EducationSpendig', 'SuperfundRightToSue', 'Crime', 'DutyFreeExports', 'ExportAdministrationActSouthAfrica']

base_normal = pd.read_csv(path, header=None, names=column_names)
base_normal = base_normal.rename(columns={"ClassName": "Decision"})

# counts = base_normal.drop(columns=['Decision']).apply(lambda col: col.value_counts()).fillna(0)
# counts

#BASE MODA GERAL
def substituir_interrogacao(col):
    if '?' in col.values:
        mais_frequente = col[col != '?'].mode()[0]  # Pega o valor mais comum ('y' ou 'n')
        col = col.replace('?', mais_frequente)
    return col


base_moda_geral=base_normal.copy()
base_moda_geral.iloc[:, 1:] = base_normal.iloc[:, 1:].apply(substituir_interrogacao)

# BASE MODA POR CLASSE
base_moda_classe = base_normal.copy()

# Itera sobre todas as colunas (menos 'ClassName')
for col in base_normal.columns[1:]:
    for classe in base_normal['Decision'].unique():
        # Filtra a coluna por classe, excluindo os '?'
        moda = base_normal.loc[(base_normal['Decision'] == classe) & (base_normal[col] != '?'), col].mode()
        if not moda.empty:
            valor_moda = moda[0]
            # Substitui na cópia onde a classe é a atual e o valor é '?'
            base_moda_classe.loc[(base_moda_classe['Decision'] == classe) & (base_moda_classe[col] == '?'), col] = valor_moda

# Resultado final
# base_moda_classe







,Decision,HandicappedInfants,WaterProjectCostSharing,AdoptionOfTheBudgetResolution,PhysicianFeeFreeze,ElSalvadorAid,ReligiousGroupsInSchools,AntiSatelliteTestBan,AidToNicaraguanContracts,MxMissile,Immigration,SynfuelsCorporationCutback,EducationSpendig,SuperfundRightToSue,Crime,DutyFreeExports,ExportAdministrationActSouthAfrica
0,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
2,democrat,y,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,n,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,n,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,y,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,n,n,n,y,n,y,y,y,n,y


In [18]:
df_train, df_test = train_test_split(base_moda_classe, test_size=0.2, random_state=42, stratify=base_moda_classe["Decision"])


config = {'algorithm': 'ID3'}
print("usando a base de treinamento")
model = chef.fit(df_train, config)



print("usando a base de teste")
evaluation = chef.evaluate(model, df_test, task="test")

usando a base de treinamento
25-04-09 22:23:36 - ID3 tree is going to be built...
25-04-09 22:23:42 - -------------------------
25-04-09 22:23:42 - finished in 6.339807510375977 seconds
25-04-09 22:23:42 - -------------------------
25-04-09 22:23:42 - Evaluate train set
25-04-09 22:23:42 - -------------------------
25-04-09 22:23:42 - Accuracy: 98.85057471264368% on 348 instances
25-04-09 22:23:42 - Labels: ['democrat' 'republican']
25-04-09 22:23:42 - Confusion matrix: [[212, 2], [2, 132]]
25-04-09 22:23:42 - Precision: 99.0654%, Recall: 99.0654%, F1: 99.0654%
usando a base de teste
25-04-09 22:23:42 - -------------------------
25-04-09 22:23:42 - Evaluate test set
25-04-09 22:23:42 - -------------------------
25-04-09 22:23:42 - Accuracy: 95.40229885057471% on 87 instances
25-04-09 22:23:42 - Labels: ['republican' 'democrat']
25-04-09 22:23:42 - Confusion matrix: [[32, 2], [2, 51]]
25-04-09 22:23:42 - Precision: 94.1176%, Recall: 94.1176%, F1: 94.1176%


/usr/local/lib/python3.11/dist-packages/chefboost/commons/functions.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Prediction"] = predictions
